Berlin can be quite ugly, particularly in Winter when it's dark and there are less than 8 hours of daylight per day. The amount of daylight I get affects my mood quite a bit, that's why I love Berlin in summer with its long hours of daylight. But when exactly does it switch? And how stark is the impact of latitude for daylight hours. This got me interested.

Let's find out how many sunlight hours I get today!

In [9]:
import ephem

def daylight_hours(latitude, longitude, date):
    observer = ephem.Observer()
    observer.lat = str(latitude)
    observer.lon = str(longitude)
    observer.date = ephem.Date(date)
    sun = ephem.Sun(observer)
    try:
        sunrise = observer.next_rising(sun).datetime()
        sunset = observer.next_setting(sun).datetime()
    except ephem.AlwaysUpError:
        return 24.0  # Sun is always up (e.g., polar day)
    except ephem.NeverUpError:
        return 0.0   # Sun never rises (e.g., polar night)

    if sunset < sunrise:
        sunset += timedelta(days=1)
    daylight_duration = sunset - sunrise
    hours_of_daylight = daylight_duration.total_seconds() / 3600

    return hours_of_daylight

# Example usage
latitude = 52.52   # Berlin latitude
longitude = 13.405 # Berlin longitude
date = '2024/01/05'

sunlight_duration = daylight_hours(latitude, longitude, date)
print("Duration of Sunlight:", sunlight_duration)


Duration of Sunlight: 7.85817605


It would be great get this number also for other cities -- and to look it  up by city name directly

In [10]:
import requests

def get_lat_long(city_name):
    url = f"https://nominatim.openstreetmap.org/search?city={city_name}&format=json"
    headers = { "User-Agent": "timeywhimey/1.0 (zaphod.beeblebrox@gmail.com)" }
    response = requests.get(url, headers=headers)
    data = response.json()

    if data:
        latitude = data[0]["lat"]
        longitude = data[0]["lon"]
        return latitude, longitude
    else:
        return None, None

city = "Berlin"
latitude, longitude = get_lat_long(city)
print("Latitude:", latitude, "Longitude:", longitude)

city = "Berlin"
date = '2024/01/05'

latitude, longitude = get_lat_long(city)
sunlight_duration = daylight_hours(latitude, longitude, date)
print("Duration of Sunlight:", sunlight_duration)

Latitude: 52.510885 Longitude: 13.3989367
Duration of Sunlight: 7.859723878055555


Now we can do some plotting. How does the daylight hours change across the year?
How does it do in different cities?
Is there a time when it doesn't matter where I am, because I get the same amount of sunlight hours on every latitude?

First, we need the data in some format that is suitable.


In [11]:
from datetime import datetime, timedelta
import pandas as pd
pd.set_option('display.precision', 1)

def generate_dates_for_year(year):
    start_date = datetime(year, 3, 1)
    end_date = datetime(year + 1, 4, 1)
    
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y/%m/%d'))
        current_date += timedelta(days=1)
    
    return date_list

dates = generate_dates_for_year(2024)
cities = ['Shanghai', 'Berlin', 'Singapore', 'Buenos Aires', 'Wellington']
df = pd.DataFrame({'Date': dates})
for city in cities:
    latitude, longitude = get_lat_long(city)
    df[city] = df.Date.map(lambda date: daylight_hours(latitude, longitude, date))

df.head(5)

,Date,Shanghai,Berlin,Singapore,Buenos Aires,Wellington
0,2024/03/01,11.6,10.9,12.1,12.8,13.0
1,2024/03/02,11.6,11.0,12.1,12.8,13.0
2,2024/03/03,11.6,11.1,12.1,12.7,12.9
3,2024/03/04,11.6,11.1,12.1,12.7,12.9
4,2024/03/05,11.7,11.2,12.1,12.7,12.8


In [15]:
import altair as alt


df_long = df.melt(id_vars='Date', var_name='city', value_name='Hours of daylight')
chart = alt.Chart(df_long).mark_line(color="grey", size=1).encode(
    x='Date:T', 
    y=alt.Y('Hours of daylight:Q', scale=alt.Scale(domain=[7, 18])),  # Limit y-axis range
    detail='city:N',
    strokeDash='city:N',
).properties(
    width=700,
    height=600
)

date_of_label = "2024/06/20"
df_left = df_long[df_long.Date == date_of_label]
df_left = df_left[df_left['city'].isin(['Berlin', 'Shanghai', 'Singapore'])]
text_chart = alt.Chart(df_left).mark_text(
    align='center', dx=0, dy=-10,
    font='IM Fell English',
    fontSize=16    
).encode(
    x='Date:T',
    y='Hours of daylight:Q',
    text='city:N',
)

date_of_label = "2024/12/20"
df_right = df_long[df_long.Date == date_of_label]
df_right = df_right[df_right['city'].isin(['Wellington', 'Buenos Aires'])]
text_chart2 = alt.Chart(df_right).mark_text(
    align='center', dx=0, dy=-10,
    font='IM Fell English',  # Specify the font here
    fontSize=16     # You can also adjust the font size
).encode(
    x='Date:T',
    y='Hours of daylight:Q',
    text='city:N',
)



layered_chart = alt.layer(chart, text_chart, text_chart2).resolve_scale(
    y='shared'
)
layered_chart.configure_view(
    strokeWidth=0       # Remove border around the chart
).configure_axis(
    grid=False,        
    domain=False,
    labelFont='IM Fell English',
    labelFontSize=14,
    titleFontWeight='normal',
    titleFont='IM Fell English',
    titleFontSize=16,
    ticks=False,
).configure_axisY(
    grid=True,
).configure_text(
    fontSize=16,
    stroke='white',      # Outer color (shadow color)
    strokeWidth=1,       # Width of the shadow
    strokeOpacity=0.5,   # Opacity of the shadow
    font='IM Fell English',  # Specify the font here
).configure_legend(
    disable=True
).configure_text(
    font='IM Fell English'
)

alt.LayerChart(...)